In [1]:
import numpy as np
import cv2,os,json
import matplotlib.pyplot as plt
import os

In [2]:
with open('training.json') as f:
  train = json.load(f)

with open('test.json') as f:
  test = json.load(f)

In [3]:
def unique_label(data):
    cat=[]
    for i in range(len(data)):
        im=data[i]['objects']
        
        for j in range(len(im)):
            cat.append(im[j]['category'])
    
    cat = np.array(cat)
    print(np.unique(cat))

In [4]:
unique_label(train)

['difficult' 'gametocyte' 'leukocyte' 'red blood cell' 'ring' 'schizont'
 'trophozoite']


In [5]:
unique_label(test)

['difficult' 'gametocyte' 'red blood cell' 'ring' 'schizont' 'trophozoite']


In [6]:
relabel = lambda lab: 'uninfected' if lab in ['leukocyte', 'red blood cell'] else 'infected'

def get_coordinate(box):
    b = box['bounding_box'].copy()
    x1 = b['minimum']['c']
    y1 = b['minimum']['r']
    x2 = b['maximum']['c']
    y2 = b['maximum']['r']
    box['bbx'] = {'x1':x1,'x2':x2,'y1':y1,'y2':y2}
    del box['bounding_box']
    
def reset(data):
    index={}
    
    for i in range(len(data)):
        sample=data[i]['objects']
        ind = []
        
        for j in range(len(sample)):
            label = sample[j]['category']
            if label=="difficult":
                ind.append(j)
            else:
                data[i]['objects'][j]['category']=relabel(label)
            get_coordinate(sample[j])
                
        if len(ind):
            index[i] = ind
            
    for index_i in index:
        indice = sorted(index[index_i], reverse=True)
        for ind in indice:
            del data[index_i]['objects'][ind]

In [7]:
reset(train)
unique_label(train)

['infected' 'uninfected']


In [8]:
reset(test)
unique_label(test)

['infected' 'uninfected']


In [9]:
with open('JSON/train.json', 'w') as f:
    json.dump(train, f)

In [10]:
with open('JSON/test.json', 'w') as f:
    json.dump(test, f)